# Tutorial on hyperparameter optimization

This notebook shows several libraries that can be used for hyperparameter optimization.
Currently, it covers:
* Sklearn: https://scikit-learn.org/stable/modules/grid_search.html
* FLAML: https://microsoft.github.io/FLAML/

TODO:
* Optuna

## 0. Setup

In [40]:
!pip install FLAML
!pip install ray[tune]
!pip install interpret

from IPython.display import clear_output
clear_output()

In [56]:
import time
import numpy as np
import pandas as pd

from flaml import tune
from functools import partial
from scipy.stats import randint
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import (
    RandomizedSearchCV,
    #TimeSeriesSplit,
    train_test_split
)
from sklearn.metrics import (
    r2_score,
    mean_absolute_error
)
from interpret.glassbox import ExplainableBoostingRegressor

In [22]:
# Load the dataset
california_housing = fetch_california_housing()

# Get the features and target variable
X = california_housing.data
y = california_housing.target

In [24]:
# Data preparation
list_features = california_housing.feature_names
col_target = 'AvgHouseVal'

df_input = pd.DataFrame(X)
df_input.columns = list_features
df_input[col_target] = y
display(df_input.head())

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,AvgHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [29]:
X_train, X_test, y_train, y_test = train_test_split(
    df_input[list_features],
    df_input[col_target],
    test_size=0.2,
    random_state=1
    )
X_train, X_val, y_train, y_val = train_test_split(
    X_train,
    y_train,
    test_size=0.25,
    random_state=1
    )
print(f"Lenghts Train/Val/Test: {len(X_train), len(X_val), len(X_test)}")

Lenghts Train/Val/Test: (12384, 4128, 4128)


## 1. Tuning with Sklearn

In [ ]:
config_search_space = {
    # Parameters for searching
    'outer_bags':10,
    'inner_bags':0,
    #'learning_rate':tune.quniform(0.01, 0.9, 0.01),
    'early_stopping_rounds':randint(60, 100, 10),
    'max_leaves':randint(2, 8, 1),

    # Fixed parameters
    'interactions':0,
    'random_state':0
}

In [32]:
# CV parameter search
n_iter = 100
random_state = 59
ebm_model = ExplainableBoostingRegressor()
cv_obj = RandomizedSearchCV(
    ebm_model,
    param_distributions=config_search_space,
    n_iter=n_iter,
    scoring="r2_score",
    random_state=random_state,
    verbose=0,
    n_jobs=-1,
)
search = cv_obj.fit(X_train, y_train)
search.best_params_

## 2. Tuning with FLAML

Tutorial based on examples from [1] and [2]

References:
* [1]: https://microsoft.github.io/FLAML/docs/getting-started
* [2]: https://microsoft.github.io/FLAML/docs/reference/tune/tune/

In [51]:
config_search_space = {
    # Parameters for searching
    'outer_bags':tune.choice([10]),
    'inner_bags':tune.choice([0]),
    #'learning_rate':tune.quniform(0.01, 0.9, 0.01),
    'early_stopping_rounds':tune.qrandint(60, 100, 10),
    'max_leaves':tune.qrandint(2, 8, 1),

    # Fixed parameters
    'interactions':tune.choice([0]),
    'random_state':tune.choice([0])
}

In [52]:
def evaluate_config(
    X_train:pd.DataFrame,
    X_val:pd.DataFrame,
    y_train:pd.Series,
    y_val:pd.Series,
    config:dict
):
  """
  Evaluate hyperparameter configuration
  """
  current_time = time.time()

  # Train model
  ebm = ExplainableBoostingRegressor(**config)
  ebm.fit(X_train, y_train)

  # Get predictions
  y_pred = ebm.predict(X_val)

  # Model evaluation
  score = np.round(r2_score(y_val, y_pred), 3)

  # FLAML report
  time2eval = time.time() - current_time
  tune.report(metric2minimize=score, time2eval=time2eval)

  return {'score': score, 'config': config}

In [53]:
# Hyperparameter search
analysis = tune.run(
    partial(evaluate_config, X_train, X_val, y_train, y_val), # function for config eval
    config=config_search_space,
    metric='score',
    mode='max', # the optimization mode. 'min' or 'max'
    num_samples=-1, # the maximal number of samples to try. -1 means infinite
    time_budget_s=1200, # the time budget in seconds,
    use_ray=False
)
clear_output()
dct_best_results = analysis.best_config
print(dct_best_results)

{'outer_bags': 10, 'inner_bags': 0, 'max_leaves': 2, 'interactions': 0, 'random_state': 0}


In [54]:
# Train model
ebm = ExplainableBoostingRegressor(**dct_best_results)
ebm.fit(X_train, y_train)

ExplainableBoostingRegressor(interactions=0, max_leaves=2, outer_bags=10,
                             random_state=0)

In [55]:
# Get predictions
y_pred = ebm.predict(X_test)

# Model evaluation
r2_score_result = np.round(r2_score(y_test, y_pred), 3)
mae_result = np.round(mean_absolute_error(y_test, y_pred), 3)
dct_metrics = {
    'r2_score_result':r2_score_result,
    'mae_result':mae_result
}
print(dct_metrics)

{'r2_score_result': 0.763, 'mae_result': 0.395}
